In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px


class Person:
    def __init__(self, name: str, base_will_power: int):
        self.name = name
        self.base_will_power = base_will_power
        self.current_life_score = 50


class LifeSimulation:
    def __init__(self, people: list[Person]):
        self.people = people

    def add_person(self, person: Person):
        self.people.append(person)

    def get_people(self) -> list[Person]:
        return self.people

    def get_person_by_name(self, name: str) -> Person | None:
        for person in self.people:
            if person.name == name:
                return person
        return None

    def get_number_of_people(self) -> int:
        return len(self.people)


def main():

    people = [
        Person("Amy", 90),
        Person("Barry", 75),
        Person("Cindy", 64),
        Person("Danny", 45),
        Person("Ethan", 25),
        Person("Frank", 10),
    ]
    simulation = LifeSimulation(people)
    simulation.simlulate_life(2000)


if __name__ == "__main__":
    main()